# Загрузка данных

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transliterate import translit
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

# сохраним путь к файлам
PATH = Path('E:')

# зафиксируем параметр random state
RS = 45

# выборка стран, города которых будем использовать
COUNTRY_CODES = [
    'RU',
    'BY',
    'KG',
    'KZ',
    'AM',
    'GE',
    'RS',
    'ME',
    'TR'
]

In [2]:
# подключимся к базе данных
DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres', 
    'password': 'password', 
    'host': 'localhost',
    'port': 5432,
    'database': 'postgres',
    'query': {}
}  

engine = create_engine(URL.create(**DATABASE))

## Данные регионов

In [3]:
# загрузим данные регионов
admin1_codes_ascii = pd.read_csv(
    PATH/'admin1CodesASCII.txt',
    delimiter='\t',
    header=None,
    names=[
        'code', 
        'name',
        'name_ascii',
        'geonameid'
    ]
)
# проверим результат
display(
    admin1_codes_ascii.head(3),
    admin1_codes_ascii.sample(3, random_state=RS),
    admin1_codes_ascii.loc[admin1_codes_ascii['name_ascii'] == 'Sverdlovsk Oblast']
)

,code,name,name_ascii,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131


,code,name,name_ascii,geonameid
294,BG.62,Veliko Tarnovo,Veliko Tarnovo,864561
319,BJ.18,Zou,Zou,2390719
3749,VN.21,Kiên Giang Province,Kien Giang Province,1579008


,code,name,name_ascii,geonameid
2817,RU.71,Sverdlovsk Oblast,Sverdlovsk Oblast,1490542


In [4]:
# поместим таблицу в БД, если ранее не поместили
try:
    admin1_codes_ascii.to_sql('admin1_codes_ascii', con=engine)
except:
    None
    
# проверим результат
query = 'SELECT * FROM admin1_codes_ascii LIMIT 2'
pd.read_sql_query(query, con=engine)

,index,code,name,name_ascii,geonameid
0,0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,1,AD.05,Ordino,Ordino,3039676


## Данные городов

In [5]:
# сохраним названия столбцов из geonames
geonames_columns = [
    'geonameid',
    'name',
    'asciiname',
    'alternatenames',
    'latitude',
    'longitude',
    'feature_class',
    'feature_code',
    'country_code',
    'cc2',
    'admin1_code',
    'admin2_code',
    'admin3_code',
    'admin4_code',
    'population',
    'elevation',
    'dem',
    'timezone',
    'modification_date'
]

# загрузим данные
cities15000 = pd.read_csv(
    PATH/'cities15000.txt',
    delimiter='\t',
    header=None,
    names=geonames_columns,
    usecols=[
        'geonameid',
        'name',
        'asciiname',
        'alternatenames',
        'country_code',
        'admin1_code',
        'admin2_code',
        'admin3_code',
        'admin4_code',
        'population'
    ]
)
# проверим результат
display(
    cities15000.head(3),
    display(cities15000[cities15000['geonameid'] == 1486209])
)

,geonameid,name,asciiname,alternatenames,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population
20938,1486209,Yekaterinburg,Yekaterinburg,"Catharinoburgum,Ekaterimburgo,Ekaterinbourg,Ek...",RU,71,NaN,NaN,NaN,1495066


,geonameid,name,asciiname,alternatenames,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",AD,08,NaN,NaN,NaN,15853
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",AD,07,NaN,NaN,NaN,20430
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",AE,07,NaN,NaN,NaN,62747


None

In [6]:
# проверим таблицу на пропуски
cities15000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27127 entries, 0 to 27126
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geonameid       27127 non-null  int64 
 1   name            27127 non-null  object
 2   asciiname       27127 non-null  object
 3   alternatenames  24799 non-null  object
 4   country_code    27113 non-null  object
 5   admin1_code     27116 non-null  object
 6   admin2_code     22092 non-null  object
 7   admin3_code     8510 non-null   object
 8   admin4_code     2628 non-null   object
 9   population      27127 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 2.1+ MB


In [7]:
cities15000[cities15000['admin2_code'].isna()]     

,geonameid,name,asciiname,alternatenames,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",AD,08,NaN,NaN,NaN,15853
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",AD,07,NaN,NaN,NaN,20430
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",AE,07,NaN,NaN,NaN,62747
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",AE,05,NaN,NaN,NaN,351943
5,291696,Khawr Fakkān,Khawr Fakkan,"Fakkan,Fakkān,Khawr Fakkan,Khawr Fakkān,Khawr ...",AE,06,NaN,NaN,NaN,40677
...,...,...,...,...,...,...,...,...,...,...
27122,894701,Bulawayo,Bulawayo,"BUQ,Bulavajas,Bulavajo,Bulavejo,Bulawayo,bu la...",ZW,09,NaN,NaN,NaN,1200337
27123,895061,Bindura,Bindura,"Bindura,Bindura Town,Kimberley Reefs,Биндура",ZW,03,NaN,NaN,NaN,50400
27124,895269,Beitbridge,Beitbridge,"Bajtbridz,Bajtbridzh,Beitbridge,Beitbridzas,Be...",ZW,07,NaN,NaN,NaN,58100
27125,1085510,Epworth,Epworth,Epworth,ZW,10,NaN,NaN,NaN,123250


В таблице пропуски в country_code и admin1_code. Это помешает вывести страну города - удалим строки с этими пропусками.

In [8]:
# удалим строки с пропусками
cities15000.dropna(
    subset=['admin1_code', 'country_code'],
    inplace=True
)

# проверим результат
cities15000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27102 entries, 0 to 27126
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geonameid       27102 non-null  int64 
 1   name            27102 non-null  object
 2   asciiname       27102 non-null  object
 3   alternatenames  24777 non-null  object
 4   country_code    27102 non-null  object
 5   admin1_code     27102 non-null  object
 6   admin2_code     22092 non-null  object
 7   admin3_code     8510 non-null   object
 8   admin4_code     2628 non-null   object
 9   population      27102 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 2.3+ MB


In [9]:
# поместим таблицу в БД, если ранее не поместили
try:
    cities15000.to_sql('cities15000', con=engine)
except:
    None

# проверим результат
query = 'SELECT * FROM cities15000 LIMIT 2'
pd.read_sql_query(query, con=engine)

,index,geonameid,name,asciiname,alternatenames,country_code,admin1_code,admin2_code,admin3_code,admin4_code,population
0,0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",AD,08,None,None,None,15853
1,1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",AD,07,None,None,None,20430


In [10]:
# загрузим альтернативные написания названий городов
alternate_names_v2 = pd.read_csv(
    PATH/'alternateNamesV2.txt',
    low_memory=False,
    delimiter='\t',
    header=None,
    names=[
        'alternateNameId',
        'geonameid', 
        'isolanguage',
        'alternate_name',
        'isPreferredName',
        'isShortName',
        'isColloquial',
        'isHistoric',
        'from',
        'to'
    ],
    usecols=[
        'alternateNameId',
        'geonameid', 
        'isolanguage',
        'alternate_name',
    ]
)
# проверим результат
display(
    alternate_names_v2.head(3),
    alternate_names_v2.loc[alternate_names_v2['geonameid'] == 1486209]
)

,alternateNameId,geonameid,isolanguage,alternate_name
0,1284819,2994701,NaN,Roc Mélé
1,1284820,2994701,NaN,Roc Meler
2,4285256,3007683,NaN,Pic des Langounelles


,alternateNameId,geonameid,isolanguage,alternate_name
11969524,724159,1486209,NaN,Sverolovsk
11969525,724160,1486209,en,Ekaterinburg
11969526,724161,1486209,NaN,Sverdlovsk
11969527,1598849,1486209,de,Ekaterinburg
11969528,1598850,1486209,en,Yekaterinburg
11969529,1598851,1486209,es,Ekaterinburgo
11969530,1598852,1486209,bg,Екатеринбург
11969531,1598853,1486209,cs,Ekatěrinburg
11969532,1598854,1486209,cv,Екатеринбург
11969533,1598855,1486209,da,Jekaterinburg


Некоторые написания на разных языках идентичны. Удалим дубликаты по geonameid и alternate_name. Так мы оставим в базе одноименные города с разным geonameid.

In [11]:
# удалим дубликаты альтернативных названий
alternate_names_v2.drop_duplicates(
    subset=['geonameid', 'alternate_name'],
    inplace=True
)
# проверим результат
alternate_names_v2.loc[alternate_names_v2['geonameid'] == 1486209]

,alternateNameId,geonameid,isolanguage,alternate_name
11969524,724159,1486209,NaN,Sverolovsk
11969525,724160,1486209,en,Ekaterinburg
11969526,724161,1486209,NaN,Sverdlovsk
11969528,1598850,1486209,en,Yekaterinburg
11969529,1598851,1486209,es,Ekaterinburgo
11969530,1598852,1486209,bg,Екатеринбург
11969531,1598853,1486209,cs,Ekatěrinburg
11969533,1598855,1486209,da,Jekaterinburg
11969537,1598859,1486209,fr,Ekaterinbourg
11969540,1598862,1486209,ja,エカテリンブルク


In [12]:
# поместим таблицу в БД, если ранее не поместили
try:
    alternate_names_v2.to_sql(
        'alternate_names_v2', 
        con=engine,
        chunksize=100000   # загрузим строки батчами этого размера, чтобы избежать переполнения памяти
    )
except:
    None

# проверим результат
query = 'SELECT * FROM alternate_names_v2 LIMIT 2'
pd.read_sql_query(query, con=engine)

,index,alternateNameId,geonameid,isolanguage,alternate_name
0,0,1284819,2994701,None,Roc Mélé
1,1,1284820,2994701,None,Roc Meler


## Данные стран

In [13]:
# загрузим данные стран
countries = pd.read_csv(
    PATH/'countryInfo.txt',
    delimiter='\t',
    header=None,
    names=[
        'country_code',
        'iso_3',
        'iso_numeric',
        'fips',
        'country',
        'capital',
        'area',
        'population',
        'continent',
        'tld',
        'currency_code',
        'currency_name',
        'phone',
        'postal_code_format',
        'postal_code_regex',
        'languages',
        'geonameid',
        'neighbours',
        'equivalent_fips_code'
    ],
    usecols=[
        'country_code',
        'country',
        'languages',
        'geonameid'
    ]
)
# проверим результат
display(
    countries.head(55),
    countries.sample(5, random_state=RS)
)

,country_code,country,languages,geonameid
0,# ================================,NaN,NaN,NaN
1,#,NaN,NaN,NaN
2,#,NaN,NaN,NaN
3,# CountryCodes:,NaN,NaN,NaN
4,# ============,NaN,NaN,NaN
5,#,NaN,NaN,NaN
6,# The official ISO country code for the United...,NaN,NaN,NaN
7,#,NaN,NaN,NaN
8,# A list of dependent countries is available h...,NaN,NaN,NaN
9,# https://spreadsheets.google.com/ccc?key=pJpy...,NaN,NaN,NaN


,country_code,country,languages,geonameid
164,KE,Kenya,"en-KE,sw-KE",192950
99,CR,Costa Rica,"es-CR,en",3624060
80,BR,Brazil,"pt-BR,es,en,fr",3469034
113,EE,Estonia,"et,ru",453733
138,GR,Greece,"el-GR,en,fr",390903


In [14]:
# удалим пропуски и лишную информацию в начале таблицы
countries = countries.dropna()[1:]

# проверим результат
countries.head(3)

,country_code,country,languages,geonameid
50,AD,Andorra,ca,3041565
51,AE,United Arab Emirates,"ar-AE,fa,en,hi,ur",290557
52,AF,Afghanistan,"fa-AF,ps,uz-AF,tk",1149361


In [15]:
# поместим таблицу в БД, если ранее не поместили
try:
    countries.to_sql('countries', con=engine)
except:
    None

# проверим результат
query = 'SELECT * FROM countries LIMIT 3'
pd.read_sql_query(query, con=engine)

,index,country_code,country,languages,geonameid
0,50,AD,Andorra,ca,3041565
1,51,AE,United Arab Emirates,"ar-AE,fa,en,hi,ur",290557
2,52,AF,Afghanistan,"fa-AF,ps,uz-AF,tk",1149361


## Тестовые данные

In [16]:
# загрузим тестовый датасет
geo_test = pd.read_csv(
    PATH/'geo_test.csv',
    delimiter=';'
)

# проверим результат
geo_test.head(30)

,query,name,region,country
0,Смоленск,Smolensk,Smolensk Oblast,Russia
1,Кемерово,Kemerovo,Kuzbass,Russia
2,Бишкек,Bishkek,Bishkek,Kyrgyzstan
3,Москва,Moscow,Moscow,Russia
4,Алматы,Almaty,Almaty,Kazakhstan
5,Оренбург,Orenburg,Orenburg Oblast,Russia
6,Новосибирск,Novosibirsk,Novosibirsk Oblast,Russia
7,Кострома,Kostroma,Kostroma Oblast,Russia
8,Ёшкар-Ола,Yoshkar-Ola,Mariy-El Republic,Russia
9,Йошкар-Ола,Yoshkar-Ola,Mariy-El Republic,Russia


Все таблицы с данными, кроме небольшого тестового датасета, мы сразу поместили в базу данных PostgeSQL. Это позволит создать гибкое масштабируемое решение, удобное для заказчика. Далее обработаем данные:

# Обработка данных

## Срез данных

Для ускорения работы кода выполним срез данных по актуальным для нас странам:

In [17]:
# функция получения выборки alternate_names для определенных стран
def get_altname_query(cities_name, country_codes):
    
    # выборка для одной страны
    if isinstance(country_codes, str):
        query = f"""
        SELECT a.geonameid, 
               a.alternate_name 
        FROM alternate_names_v2 a
        JOIN {cities_name} c ON a.geonameid=c.geonameid 
        WHERE country_code = '{country_codes}'
        """
        
    # выборка для нескольких стран
    else:
        query = f"""
        SELECT a.geonameid, 
               a.alternate_name 
        FROM alternate_names_v2 a
        JOIN {cities_name} c ON a.geonameid=c.geonameid 
        WHERE country_code IN {tuple(country_codes)}
        """

    altname_query = pd.read_sql_query(query, con=engine)
    return altname_query

In [18]:
# получим выборку alternate_names в некоторых странах 
altname_query = get_altname_query('cities15000', COUNTRY_CODES)

# проверим результат
altname_query.sample(5, random_state=RS)

,geonameid,alternate_name
17765,2118647,Вілючинськ
9481,533543,Luchowizy
17951,6417459,Краснознаменськ
2575,1516589,Dzhezkazgan
5921,478071,Ust'-Katav


Создадим функцию, оторая сохранит выборку для последующих обращений к ней:

In [19]:
# функция сохранения выборки alternate_names для определенных стран в БД
def save_altname_query(altname_query, altname_query_name):
    try:
        altname_query.to_sql(altname_query_name, con=engine)
    except:
        None

## Транслитерация

Транслитерация названий городов поможет нам унифицировать данные, записав их латиницей. Это должно улучшить качество нашего алгоритма.

In [20]:
# функция транслитерации для столбца датасета
def translit_data(df):
    transliteration = []
    for row in df:
        try:
            transliteration.append(translit(row, reversed=True))
        except:
            transliteration.append(row)
    return transliteration

In [21]:
%%time
# сделаем транслитерацию среза данных
altname_query['alternate_name'] = translit_data(altname_query['alternate_name'])
altname_query.head()

Wall time: 4.34 s


,geonameid,alternate_name
0,174875,Qafan
1,174875,Kapan
2,174875,کاپان
3,174875,Kapan
4,174875,Kapan


In [22]:
%%time
# сохраним выборку
save_altname_query(altname_query, 'altname_query')

# проверим результат
query = 'SELECT * FROM altname_query LIMIT 3'
pd.read_sql_query(query, con=engine)

Wall time: 631 ms


,index,geonameid,alternate_name
0,0,174875,Qafan
1,1,174875,Kapan
2,2,174875,کاپان


## Векторизация

Используем посимвольную векторизацию TF-IDF с би-граммами и три-граммами букв. N-граммы помогут улучшить обработку запросов с опечатками и ошибками.

In [23]:
# функции векторизации TF-IDF 
def tfidf_fit_transform(ngram_range, data):
    tf_idf_vectorizer = TfidfVectorizer(
        analyzer='char', 
        lowercase=True,
        ngram_range=ngram_range
    )
    vector = tf_idf_vectorizer.fit_transform(data)
    
    return vector, tf_idf_vectorizer

def tfidf_transform(query, tf_idf_vectorizer):
    vec_query = tf_idf_vectorizer.transform([query])
    return vec_query

In [24]:
%%time
# проверим функции векторизации TF-IDF
vector, tf_idf_vectorizer = tfidf_fit_transform((1, 3), altname_query['alternate_name'])
vec_query = tfidf_transform('Ekaterinburg', tf_idf_vectorizer)
display(
    vector,
    vec_query
)

<21791x31589 sparse matrix of type '<class 'numpy.float64'>'
	with 644378 stored elements in Compressed Sparse Row format>

<1x31589 sparse matrix of type '<class 'numpy.float64'>'
	with 31 stored elements in Compressed Sparse Row format>

Wall time: 746 ms


Результат векторизации оставим в виде разряженных матриц. Так мы сможет быстро обрабатывать большой объем векторов значительного размера. Наши матрицы мало весят, создаются достаточно быстро и не требуют специального хранения в БД.

Мы успешно создали срез данных для ускорения вычислений и провели транслитерацию. Мы сохранили часть результов обработки в БД и проверили векторизатор. Обработка данных успешно завершена. Приступим к поиску похожих названий городов.

# Поиск похожих названий

Изначально для поиска схожих названий мы использовали FAISS. Индекс FAISS может принимать на вход массивы, но не разряженные матрицы. При увеличении размера данных векторы TF-IDF в виде массива вызывали переполнение памяти. Мы решили заменить FAISS на поиск cosine_similarity из библиотеки scikit-learn:

In [25]:
# функция поиска похожих названий городов
def get_similar_tfidf(query, num, cities_name, admin1_codes_ascii_name, countries_name, altname_query_name, country_codes, alternate_name):
    """
    Функция принимает:
    - query - запрос с названием города
    - num - количество выводимых похожих названий городов из geonames
    - cities_name - название таблицы с данными городов в БД
    - admin1_codes_ascii_name - название таблицы с данными регионов в БД
    - countries_name - название таблицы с данными стран в БД
    - altname_query_name - название выборки написаний названий городов для определенных стран в БД
    - country_codes - коды стран из выборки
    - alternate_name - название поля с альтернативными написаниями городов из таблицы altname_query_name в БД
    Функция создает срез данных по странам, если его еще нет в БД и сохраняет срез в БД. Функция транслитерирует запрос,
    находит cosine similarity похожих названий городов и их geonameid. Данные обогащаются регионом и страной.
    Города сортируются по cosine similarity и затем по населению.
    Функция возвращает geonameid похожих городов, названия городов из geonames, регион, страну, cosine similarity.
    """
    # получим выборку alternate_names в некоторых странах
    try:
        query_altname = f'SELECT * FROM {altname_query_name}'
        altname_query = pd.read_sql_query(query_altname, con=engine)
    # если выборку не делали, сформируем и сохраним ее    
    except:
        altname_query = get_altname_query(cities_name, country_codes)
        altname_query[alternate_name] = translit_data(altname_query[alternate_name])
        save_altname_query(altname_query, altname_query_name)

    # транслитерируем запрос
    try:
        query = translit(query, reversed=True)
    except:
        None
    # получим векторы альтернативных названий и вектор запроса
    vector, tf_idf_vectorizer = tfidf_fit_transform((1, 3), altname_query[alternate_name])
    vec_query = tfidf_transform(query, tf_idf_vectorizer)

    # найдем индексы и cosine similarity похожих названий, ограничим количество в выводе
    cos_sim = cosine_similarity(vector, vec_query)     
    idx = cos_sim.argsort(axis=None)[::-1][:num*10]    
    dist = np.sort(cos_sim, axis=None)[::-1][:num*10]
    
    similar = pd.DataFrame(altname_query.loc[idx, 'geonameid'])
    similar['cosine_similarity'] = dist
    similar = similar.drop_duplicates(subset='geonameid').reset_index(drop=True)

    # найдем названия городов, коды стран и регионов и население
    if len(similar) != 1:
        query_cities = f"""
        SELECT * 
        FROM 
        (SELECT geonameid, 
                asciiname name, 
                country_code code_from_cities, 
                admin1_code, 
                population 
         FROM {cities_name} 
         WHERE geonameid IN {tuple(similar['geonameid'])}) AS ci 
         LEFT OUTER JOIN 
        (SELECT country, 
                country_code 
        FROM {countries_name}) AS co ON co.country_code = ci.code_from_cities
        """
    else:
        query_cities = f"""
        SELECT * 
        FROM 
        (SELECT geonameid, 
                asciiname name, 
                country_code code_from_cities, 
                admin1_code, 
                population 
         FROM {cities_name} 
         WHERE geonameid = {similar['geonameid'][0]}) AS ci 
         LEFT OUTER JOIN 
        (SELECT country, 
                country_code 
        FROM {countries_name}) AS co ON co.country_code = ci.code_from_cities
        """
    # добавим столбцы из выборки
    similar = pd.merge(
        similar, 
        pd.read_sql_query(query_cities, con=engine), 
        on='geonameid'
    )
    similar['code'] = similar['country_code'] + '.' + similar['admin1_code']

    # добавим регион
    if len(similar) != 1:
        query_region_country = f"""
        SELECT name_ascii region, 
               code 
        FROM {admin1_codes_ascii_name} 
        WHERE code IN {tuple(similar['code'])}
        """
    else:
        query_region_country = f"""
        SELECT name_ascii region, 
               code 
        FROM {admin1_codes_ascii_name} 
        WHERE code = '{similar['code'][0]}'
        """
    similar = pd.merge(
        similar, 
        pd.read_sql_query(query_region_country, con=engine), 
        on='code'
    )  
    # отсортируем города по cosine_similarity и затем по населению
    similar = similar.sort_values(
        by=['cosine_similarity', 'population'],
        ascending=False
    )
    # уберем ненужные строки
    similar = similar[:num]
    
    # расположим столбцы в нужном порядке
    similar = similar[['geonameid', 'name', 'region', 'country', 'cosine_similarity']].reset_index(drop=True)
    
    return similar

In [26]:
%%time
# найдем похожие названия городов
similar = get_similar_tfidf(
    'Ираславль', 
    5, 
    'cities15000', 
    'admin1_codes_ascii',
    'countries',
    'altname_query',
    COUNTRY_CODES,
    'alternate_name'
)
similar

Wall time: 955 ms


,geonameid,name,region,country,cosine_similarity
0,501283,Roslavl',Smolensk Oblast,Russia,0.706121
1,468902,Yaroslavl,Yaroslavl Oblast,Russia,0.638983
2,530849,Maloyaroslavets,Kaluga Oblast,Russia,0.472458
3,511359,Pereslavl'-Zalesskiy,Yaroslavl Oblast,Russia,0.419353
4,473247,Vladimir,Vladimir Oblast,Russia,0.362102


Алгоритм не всегда качественно обрабатывает опечатки и ошибки, особенно если опечаток несколько. Тем не менее верный город часто оказывается в выборке, пусть и не на первом месте. Проверим как алгоритм выводит одноименные города, расположенные в разных регионах:

In [27]:
%%time
# найдем похожие названия городов
similar = get_similar_tfidf(
    'Киров', 
    5, 
    'cities15000', 
    'admin1_codes_ascii',
    'countries',
    'altname_query',
    COUNTRY_CODES,
    'alternate_name'
)
similar

Wall time: 785 ms


,geonameid,name,region,country,cosine_similarity
0,548408,Kirov,Kirov Oblast,Russia,1.000000
1,548410,Kirov,Kaluga Oblast,Russia,1.000000
2,548391,Kirovsk,Murmansk,Russia,0.846202
3,548392,Kirovsk,Leningradskaya Oblast',Russia,0.846202
4,299445,Tekirova,Antalya,Turkey,0.671288


На первом месте в таблице более крупный город.

По просьбе заказчика результатом работы алгоритма должен быть список словарей - напишем соответствующую функцию:

In [28]:
# функция преобразует датасет в список словарей
def get_dicts(df):
    dicts = df.to_dict(orient='records')
    return dicts

# проверим результат
similar_dicts = get_dicts(similar)
similar_dicts

[{'geonameid': 548408,
  'name': 'Kirov',
  'region': 'Kirov Oblast',
  'country': 'Russia',
  'cosine_similarity': 1.0},
 {'geonameid': 548410,
  'name': 'Kirov',
  'region': 'Kaluga Oblast',
  'country': 'Russia',
  'cosine_similarity': 1.0},
 {'geonameid': 548391,
  'name': 'Kirovsk',
  'region': 'Murmansk',
  'country': 'Russia',
  'cosine_similarity': 0.8462021298837237},
 {'geonameid': 548392,
  'name': 'Kirovsk',
  'region': "Leningradskaya Oblast'",
  'country': 'Russia',
  'cosine_similarity': 0.8462021298837237},
 {'geonameid': 299445,
  'name': 'Tekirova',
  'region': 'Antalya',
  'country': 'Turkey',
  'cosine_similarity': 0.6712884705078531}]

Мы создали рабочее решение, которое ищет похожие названия городов из geonames. Функции выводят нужные заказчику поля в правильном формате.

Алгоритм не лишен недостатков - далее тщательнее проверим качество решения.

# Проверка качества решения

Проверим наше решение на данных из тестового датасета:

In [29]:
def check_accuracy(test):
    """
    Функция принимает тестовый датасет,
    ищет похожие города и добавляет к датасету справа:
    - geonameid
    - название города
    - регион и страну
    - cosine_similarity
    Также функция считает и выводит accuracy по названию города.
    """
    preds = []
    for query in test['query']:
        preds.append(
            get_dicts(
                get_similar_tfidf(
                    query, 
                    1, 
                    'cities15000', 
                    'admin1_codes_ascii',
                    'countries',
                    'altname_query',
                    COUNTRY_CODES,
                    'alternate_name'
                )
            )
        )
    preds = pd.DataFrame(sum(preds, []))
    preds.columns = ['geonameid_preds', 'name_preds', 'region_preds', 'country_preds', 'cosine_similarity']
    
    # соединим тестовые данные и предсказания
    result = pd.concat([test, preds], axis=1)
    accuracy = (test['name'] == preds['name_preds']).sum() / len(test)
    print(f'Accuracy на тестовой выборке: {"%.3f" % (accuracy)}')
    
    return result

In [30]:
%%time
# проверим качество нашего решения
check_df = check_accuracy(geo_test)
check_df.sample(15, random_state=RS)

Accuracy на тестовой выборке: 0.871
Wall time: 4min 2s


,query,name,region,country,geonameid_preds,name_preds,region_preds,country_preds,cosine_similarity
202,Первоуральск,Pervouralsk,Sverdlovsk Oblast,Russia,510808,Pervouralsk,Sverdlovsk Oblast,Russia,1.000000
112,Саранск,Saransk,Mordoviya Republic,Russia,498698,Saransk,Mordoviya Republic,Russia,1.000000
20,Нижний Новгород,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,520555,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,1.000000
4,Алматы,Almaty,Almaty,Kazakhstan,1526384,Almaty,Almaty,Kazakhstan,1.000000
194,Невиномыск,Nevinnomyssk,Stavropol Kray,Russia,522377,Nevinnomyssk,Stavropol Kray,Russia,1.000000
10,Минск,Minsk City,Minsk City,Belarus,625144,Minsk,Minsk City,Belarus,1.000000
63,Ираславль,Yaroslavl,Yaroslavl Oblast,Russia,501283,Roslavl',Smolensk Oblast,Russia,0.706121
122,Шымкент,Shymkent,Shymkent,Kazakhstan,1518980,Shymkent,Shymkent,Kazakhstan,1.000000
176,Актау,Shevchenko,Mangghystaū,Kazakhstan,610612,Shevchenko,Mangghystau,Kazakhstan,1.000000
319,Елец,Yelets,Lipetsk Oblast,Russia,467978,Yelets,Lipetsk Oblast,Russia,1.000000


Accuracy неидельная, но на приемлемом уровне. Подробнее взглянем на ошибки нашего алгоритма:

In [31]:
check_df[check_df['name'] != check_df['name_preds']].head(15)

,query,name,region,country,geonameid_preds,name_preds,region_preds,country_preds,cosine_similarity
10,Минск,Minsk City,Minsk City,Belarus,625144,Minsk,Minsk City,Belarus,1.000000
14,Рязань,Ryazan’,Ryazan Oblast,Russia,500096,Ryazan',Ryazan Oblast,Russia,1.000000
15,Екб,Yekaterinburg,Sverdlovsk Oblast,Russia,1524325,Ekibastuz,Pavlodar Region,Kazakhstan,1.000000
17,Н.Новгород,Nizhniy Novgorod,Nizhny Novgorod Oblast,Russia,519336,Velikiy Novgorod,Novgorod Oblast,Russia,0.967414
37,Орел,Orël,Oryol oblast,Russia,515012,Orel,Oryol oblast,Russia,1.000000
44,Островцы,Ostrovtsy,Moscow Oblast,Russia,514171,Ostrov,Pskov Oblast,Russia,0.762172
54,Солегорск,Salihorsk,Minsk,Belarus,515698,Olenegorsk,Murmansk,Russia,0.725415
63,Ираславль,Yaroslavl,Yaroslavl Oblast,Russia,501283,Roslavl',Smolensk Oblast,Russia,0.706121
67,Влодевасток,Vladivostok,Primorye,Russia,300619,Sivas,Sivas,Turkey,0.410467
78,Назрань,Nazran’,Ingushetiya Republic,Russia,523064,Nazran',Ingushetiya Republic,Russia,1.000000


Наше решение неважно распознает сокращения и множественные опечатки. Улучшить качество поможет аугментация данных. Также будет полезно протестировать эмбеддинги и дообучение языковых моделей. В сочетании с ними будут полезны инструменты FAISS + k-means кластеризация для ускорения вычислений. 

Отметим ошибки в тестовом датасете - среди городов есть страны, а некоторые наименования не соответствуют стандарту geonames (другие языки, нестандартные названия).

# Вывод

1. Все таблицы с данными, кроме небольшого тестового датасета, мы поместили в базу данных PostgeSQL. Это позволило создать гибкое масштабируемое решение, удобное для заказчика. В БД также хранится срез данных с вариантами написания названий городов, необходимый для быстрой работы алгоритма

2. Мы успешно создали срез данных для ускорения вычислений и провели транслитерацию. Мы сохранили часть результов обработки в БД и проверили векторизатор. Успешно завершили обработку данных.

3. Результат векторизации мы оставили в виде разряженных матриц. Так мы смогли быстро обрабатывать большой объем векторов значительного размера. Наши матрицы мало весят, создаются достаточно быстро и не требуют специального хранения в БД.

4. Мы создали рабочее решение, которое ищет похожие названия городов из geonames:
   - Функция создает срез данных по странам, если его еще нет в БД и сохраняет срез в БД. Функция транслитерирует запрос, находит cosine similarity похожих названий городов и их geonameid. Данные обогащаются регионом и страной. Города сортируются по cosine similarity и затем по населению. Функция возвращает geonameid похожих городов, названия городов из geonames, регион, страну, cosine similarity.
   - Другая функция возвращает поля в требуемом формате списка словарей.

5. На тестовых данных алгоритм показал accuracy на уровне 0.871. Мы использовали самый строгий расчет точности по первому выдаваемому варианту. Качество неидельное, но приемлемое. Можно добавить расчет accuracy по n-выдаче алгоритма, считая за правильный ответ нужный вариант на любой позиции в n-выдаче. 

6. Наше решение неважно распознает сокращения и множественные опечатки. Улучшить качество поможет аугментация данных. Также будет полезно протестировать эмбеддинги и дообучение языковых моделей. В сочетании с ними будут полезны инструменты FAISS + кластеризация k-means для ускорения вычислений. 

7. Отметим ошибки в тестовом датасете - среди городов есть страны, а некоторые наименования не соответствуют стандарту geonames (другие языки, нестандартные названия).

**Итого:**
Мы создали рабочее решение с приемлемым качеством и скоростью работы. Но алгоритм можно доработать, применив аугментацию, эмбеддинги названий городов и дообучение языковых моделей. 